In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from numpy.typing import NDArray
from scipy.optimize import minimize
from typing import List, Tuple
import time as time
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
# 価格を生成する関数
def create_price(r_mean: float, r_std: float, M: int) -> NDArray[np.float_]:
    # r_mean = (r_min + r_max) / 2
    # r_std = (r_max - r_mean) / 2
    # r_minとr_maxの間のランダムな0.1刻みの少数をM個生成
    price = np.random.normal(r_mean, r_std, size=M)
    # price = np.round(price, 1)

    return price


# alphaを作成する関数
def alpha_star(M: int) -> NDArray[np.float_]:
    alpha_star = np.random.uniform(M, 3 * M, size=M)
    return alpha_star


# betaを作成する関数
def beta_star(M: int, M_prime: int) -> NDArray[np.float_]:
    beta_star = np.zeros((M, M_prime))

    for m in range(M):
        for m_prime in range(M_prime):
            if m == m_prime:
                beta_star[m, m_prime] = np.random.uniform(-3 * M, -2 * M)
            else:
                beta_star[m, m_prime] = np.random.uniform(0, 3)

    return beta_star


def quantity_function(
    price: NDArray[np.float_],
    alpha: NDArray[np.float_],
    beta: NDArray[np.float_],
    delta: float = 0.1,  # ノイズレベルを指定（例として0.1を使用）
) -> list[float]:
    M = len(price)
    quantity_list = []
    q_m_no_noise = []

    # ステップ1: ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]
        quantity = alpha[m] + sum_beta
        q_m_no_noise.append(quantity)

    # E[q_m^2]を計算
    E_q_m_squared = np.mean(np.array(q_m_no_noise) ** 2)

    # ステップ2: ノイズの標準偏差sigmaを計算
    sigma = delta * np.sqrt(E_q_m_squared)

    # ステップ3: ノイズを加えて最終的なq_mを計算
    for m in range(M):
        epsilon = np.random.normal(0, sigma)
        quantity = q_m_no_noise[m] + epsilon
        quantity_list.append(quantity)

    return quantity_list


def sales_function(
    price: NDArray[np.float_], alpha: NDArray[np.float_], beta: NDArray[np.float_]
) -> list[float]:
    M = len(price)
    sales_list = []

    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]

        quantity = alpha[m] + sum_beta
        sales_list.append(quantity * price[m])

    return sales_list


def create_date(M, N, r_mean, r_std, delta=0.1):
    alpha = alpha_star(M)
    beta = beta_star(M, M)

    price_list = []
    quantity_list = []

    for _ in range(N):
        price = create_price(r_mean, r_std, M)
        quantity = quantity_function(price, alpha, beta, delta)
        price_list.append(price)
        quantity_list.append(quantity)

    X = np.array(price_list)
    Y = np.array(quantity_list)

    return alpha, beta, X, Y


def create_bounds(M, r_min, r_max):
    lb = np.full(M, r_min)
    ub = np.full(M, r_max)

    range_bounds = []
    for i in range(M):
        range_bounds.append(lb[i])

    for i in range(M):
        range_bounds.append(ub[i])

    bounds = [(r_min, r_max) for _ in range(M)]

    return lb, ub, bounds, range_bounds

In [3]:
# 目的関数を定義（最大化問題を最小化問題に変換）
def sales_objective_function(prices, alpha, beta, M):
    return -sum(
        prices[m] * (alpha[m] + sum(beta[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


def sales_optimize(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    bounds: list[tuple[float, float]],
) -> Tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)

    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [4]:
# 目的関数を定義
def predict_objective_function(
    prices: NDArray[np.float_], intercepts: [float], coefs: [NDArray[np.float_]], M: int
) -> float:
    # 各変数の内容をデバッグ出力
    # print("prices:", prices)
    # print("intercepts:", intercepts)
    # print("coefs:", coefs)
    # print("M:", M)

    return -sum(
        prices[m]
        * (intercepts[m] + sum(coefs[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


# 予測と最適化を行う関数
def predict_optimize(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], bounds: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [5]:
# CVを行う関数
def cross_validation(
    tilda_coefs_list: list[NDArray[np.float_]],
    tilda_intercepts_list: list[float],
    hat_coefs_list: list[NDArray[np.float_]],
    hat_intercepts_list: list[float],
    M: int,
    K: int,
    bounds: list[float],
) -> float:
    optimal_sales_list = []
    optimal_prices_list = [[] for _ in range(M)]
    for i in range(K):
        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)

        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(tilda_intercepts_list[i], tilda_coefs_list[i], M),
            bounds=bounds,
            method="L-BFGS-B",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        # print("optimal_prices cv:", optimal_prices)
        for m in range(M):
            optimal_prices_list[m].append(optimal_prices[m])

        sales_hat = np.sum(
            sales_function(optimal_prices, hat_intercepts_list[i], hat_coefs_list[i])
        )

        optimal_sales_list.append(sales_hat)

    return np.mean(optimal_sales_list), optimal_prices_list

In [6]:
def cross_validation_bounds_penalty_all(
    bounds: List[float],
    tilda_coefs_list: List[NDArray[np.float_]],
    tilda_intercepts_list: List[NDArray[np.float_]],
    hat_coefs_list: List[NDArray[np.float_]],
    hat_intercepts_list: List[NDArray[np.float_]],
    M: int,
    K: int,
    bounds_range: float,
    lamda: float,
) -> float:
    # bounds の整合性チェック
    bounds_list = []
    for i in range(M):
        if bounds[i] > bounds[i + M]:
            bounds_mean = (bounds[i] + bounds[i + M]) / 2
            bounds_list.append((bounds_mean, bounds_mean))
        else:
            bounds_list.append((bounds[i], bounds[i + M]))
    optimal_sales_list = []

    # すでに外部でKFold分割や学習が終わっているものとして
    # tilda_coefs_list[i], tilda_intercepts_list[i], hat_coefs_list[i], hat_intercepts_list[i]
    # を使用して最適化と売上計算
    for i in range(K):
        intercepts = tilda_intercepts_list[i]
        coefs = tilda_coefs_list[i]

        # 最適化
        initial_prices = np.full(M, 0.6)
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds_list,
            method="L-BFGS-B",
        )
        optimal_prices = result.x

        # hatモデルパラメータで売上計算
        alpha = hat_intercepts_list[i]
        beta = hat_coefs_list[i]

        sales_hat = np.sum(sales_function(optimal_prices, alpha, beta))
        optimal_sales_list.append(sales_hat)

    # ペナルティ計算
    penalty = 0.0
    for i in range(M):
        penalty += bounds[i + M] - bounds[i]

    mean_sales = np.mean(optimal_sales_list)

    return -mean_sales + lamda * max(0, penalty - M * bounds_range) ** 2


def estimate_bounds_penalty_nelder_all(
    bounds: List[float],
    tilda_coefs_list: List[NDArray[np.float_]],
    tilda_intercepts_list: List[NDArray[np.float_]],
    hat_coefs_list: List[NDArray[np.float_]],
    hat_intercepts_list: List[NDArray[np.float_]],
    M: int,
    K: int,
    r_min: float,
    r_max: float,
    bounds_range: float,
    lamda: float,
    adaptive: bool = True,
) -> Tuple[float, List[Tuple[float, float]]]:
    # Nelder-Meadでの最適化
    bounds_nelder = minimize(
        cross_validation_bounds_penalty_all,
        bounds,
        args=(
            tilda_coefs_list,
            tilda_intercepts_list,
            hat_coefs_list,
            hat_intercepts_list,
            M,
            K,
            bounds_range,
            lamda,
        ),
        method="Nelder-Mead",
        bounds=[(r_min, r_max) for _ in range(2 * M)],
        options={"adaptive": adaptive},
    )

    opt_bounds = []
    for i in range(M):
        lb = min(bounds_nelder.x[i], bounds_nelder.x[i + M])
        ub = max(bounds_nelder.x[i], bounds_nelder.x[i + M])
        opt_bounds.append((lb, ub))

    return -bounds_nelder.fun, opt_bounds

In [7]:
def bound_quan(price_list: np.ndarray, q: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    price_list : shape (N, M) の価格データ (N 件のサンプル、M 商品分)
    q          : 上限とする分位数 (例: 0.95 など)

    戻り値:
        lower_bound : (1-q) 分位数 (shape (M,))
        upper_bound : q 分位数   (shape (M,))
    """
    # axis=0 で列ごとに分位数を計算
    lower_bound = np.quantile(price_list, q, axis=0)
    upper_bound = np.quantile(price_list, 1 - q, axis=0)
    bounds_quan = []
    for i in range(len(lower_bound)):
        bounds_quan.append((lower_bound[i], upper_bound[i]))
    return bounds_quan

In [9]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 0.6
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []

quan_sales_list_5 = []
true_quan_sales_list_5 = []

ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    np.random.seed(i + 6)
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # QUAN
    quan_bounds = bound_quan(X, 0.1)
    quan_po_sales, quan_po_prices = predict_optimize(M, X, Y, quan_bounds)
    true_quan_po_sales = np.sum(sales_function(quan_po_prices, alpha, beta))
    quan_sales_list_5.append(quan_po_sales / so_sales)
    true_quan_sales_list_5.append(true_quan_po_sales / so_sales)

    # EBZ
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)

    # EBPA(0.3)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)

    # EBPA(0.4)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)

    # EBPA(0.5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [10]:
df_5_6 = pd.DataFrame(
    {
        "SO": so_sales_list_5,
        "PO": po_sales_list_5,
        "True PO": true_po_sales_list_5,
        "QUAN": quan_sales_list_5,
        "True QUAN": true_quan_sales_list_5,
        "EBZ": ebz_po_sales_list_5,
        "True EBZ": true_ebz_po_sales_list_5,
        "EBPA(0.3)": ebpa3_po_sales_list_5,
        "True EBPA(0.3)": true_ebpa3_po_sales_list_5,
        "EBPA(0.4)": ebpa4_po_sales_list_5,
        "True EBPA(0.4)": true_ebpa4_po_sales_list_5,
        "EBPA(0.5)": ebpa5_po_sales_list_5,
        "True EBPA(0.5)": true_ebpa5_po_sales_list_5,
        "EBZ Range": ebz_range_deff_list_5,
        "EBPA(0.3) Range": ebpa3_range_deff_list_5,
        "EBPA(0.4) Range": ebpa4_range_deff_list_5,
        "EBPA(0.5) Range": ebpa5_range_deff_list_5,
    }
)

In [11]:
df_5_6

,SO,PO,True PO,QUAN,True QUAN,EBZ,True EBZ,EBPA(0.3),True EBPA(0.3),EBPA(0.4),True EBPA(0.4),EBPA(0.5),True EBPA(0.5),EBZ Range,EBPA(0.3) Range,EBPA(0.4) Range,EBPA(0.5) Range
0,1.0,1.065025,0.945590,1.035588,0.974942,1.036994,0.968714,1.037070,0.968751,1.036174,0.965550,1.036220,0.965622,2.026702,1.207073,1.913242,1.993571
1,1.0,0.982205,0.982429,0.963393,0.986516,0.960388,0.965019,0.950286,0.957594,0.978959,0.973910,0.960390,0.965017,2.355169,0.446006,1.217866,2.287704
2,1.0,0.961540,0.942605,0.961179,0.944472,0.955741,0.950682,0.943744,0.927158,0.940133,0.935589,0.951983,0.942140,2.396336,0.668584,1.223039,1.886251
3,1.0,1.070680,0.957278,0.983971,0.935289,1.049916,0.974516,1.036657,0.952445,1.037906,0.947539,1.049910,0.974514,2.568438,1.500509,1.710495,2.446303
4,1.0,0.990095,0.925702,0.980126,0.976651,0.948613,0.970712,0.944524,0.973684,0.944519,0.973679,0.948617,0.970716,2.153974,1.420944,1.551240,1.964250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,1.034478,0.983472,1.017353,0.967867,1.034350,0.982862,1.027678,0.972946,1.029181,0.969768,1.034350,0.982862,2.836479,1.202223,0.158084,2.215237
96,1.0,1.070770,0.927718,1.038250,0.952181,1.069511,0.935296,1.067377,0.933622,1.068838,0.942809,1.067880,0.934467,2.565701,0.882159,1.965411,0.622522
97,1.0,1.030068,0.973713,1.014726,0.980588,1.025846,0.982344,1.024103,0.960469,1.024097,0.960466,1.022320,0.963937,2.416603,0.768603,1.127153,0.739322
98,1.0,1.087754,0.986568,1.012813,0.975147,1.078440,0.985234,1.075546,0.986845,1.078250,0.985911,1.077903,0.985424,2.587232,1.495055,1.942276,2.470592


In [12]:
df_5_6_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "EBZ Range": each_range_deff_6_list,
        "EBPA(0.3) Range": each_range_deff_3_list,
        "EBPA(0.4) Range": each_range_deff_4_list,
        "EBPA(0.5) Range": each_range_deff_5_list,
    }
)

In [13]:
df_5_6_each

,R2,EBZ Range,EBPA(0.3) Range,EBPA(0.4) Range,EBPA(0.5) Range
0,0.116281,0.411202,0.377197,0.408085,0.430116
1,0.073423,0.300877,0.291760,0.313975,0.307020
2,0.126188,0.378241,0.395806,0.389789,0.390076
3,0.071567,0.462477,0.055757,0.375777,0.399979
4,0.129011,0.473905,0.086553,0.425616,0.466379
...,...,...,...,...,...
495,0.154530,0.487887,0.122658,0.349781,0.470095
496,0.130302,0.565113,0.323969,0.415453,0.537654
497,0.090216,0.344138,0.323458,0.321588,0.348597
498,0.150955,0.560332,0.227340,0.422247,0.541822


In [14]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 0.8
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []

quan_sales_list_5 = []
true_quan_sales_list_5 = []

ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    np.random.seed(i + 8)
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # QUAN
    quan_bounds = bound_quan(X, 0.1)
    quan_po_sales, quan_po_prices = predict_optimize(M, X, Y, quan_bounds)
    true_quan_po_sales = np.sum(sales_function(quan_po_prices, alpha, beta))
    quan_sales_list_5.append(quan_po_sales / so_sales)
    true_quan_sales_list_5.append(true_quan_po_sales / so_sales)

    # EBZ
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)

    # EBPA(0.3)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)

    # EBPA(0.4)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)

    # EBPA(0.5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [15]:
df_5_8 = pd.DataFrame(
    {
        "SO": so_sales_list_5,
        "PO": po_sales_list_5,
        "True PO": true_po_sales_list_5,
        "QUAN": quan_sales_list_5,
        "True QUAN": true_quan_sales_list_5,
        "EBZ": ebz_po_sales_list_5,
        "True EBZ": true_ebz_po_sales_list_5,
        "EBPA(0.3)": ebpa3_po_sales_list_5,
        "True EBPA(0.3)": true_ebpa3_po_sales_list_5,
        "EBPA(0.4)": ebpa4_po_sales_list_5,
        "True EBPA(0.4)": true_ebpa4_po_sales_list_5,
        "EBPA(0.5)": ebpa5_po_sales_list_5,
        "True EBPA(0.5)": true_ebpa5_po_sales_list_5,
        "EBZ Range": ebz_range_deff_list_5,
        "EBPA(0.3) Range": ebpa3_range_deff_list_5,
        "EBPA(0.4) Range": ebpa4_range_deff_list_5,
        "EBPA(0.5) Range": ebpa5_range_deff_list_5,
    }
)


df_5_8

,SO,PO,True PO,QUAN,True QUAN,EBZ,True EBZ,EBPA(0.3),True EBPA(0.3),EBPA(0.4),True EBPA(0.4),EBPA(0.5),True EBPA(0.5),EBZ Range,EBPA(0.3) Range,EBPA(0.4) Range,EBPA(0.5) Range
0,1.0,0.972994,0.905940,0.970978,0.919581,0.947565,0.928479,0.939177,0.915275,0.940432,0.913920,0.940860,0.914520,1.457729,0.645709,1.095284,0.786665
1,1.0,1.111928,0.934514,1.003141,0.914660,1.076324,0.953027,1.058065,0.925673,1.075184,0.959258,1.076328,0.953027,2.586977,0.941759,1.871808,2.438254
2,1.0,1.012859,0.916143,0.981692,0.973814,0.930115,0.945971,0.930066,0.945934,0.930104,0.945967,0.930126,0.945957,1.732360,1.482479,1.567213,1.899927
3,1.0,0.968383,0.983142,0.900843,0.915151,0.968383,0.983142,0.965564,0.983236,0.965503,0.983104,0.967320,0.984840,2.937840,0.478622,0.583112,0.980805
4,1.0,1.008070,0.989088,0.978340,0.967612,1.005060,0.993437,1.002627,0.988269,1.003455,0.997689,1.005060,0.993437,2.686191,0.359106,1.733816,2.500309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,1.051344,0.956273,1.027239,0.972848,1.044315,0.972427,1.038610,0.946761,1.038798,0.967951,1.038800,0.967952,2.476074,1.203826,1.352759,1.881717
96,1.0,1.122032,0.982637,1.025552,0.973841,1.105739,0.981368,1.105575,0.984414,1.106761,0.981039,1.106761,0.981040,2.564809,1.390160,1.988319,2.500181
97,1.0,1.046318,0.850791,0.930968,0.793074,1.014283,0.909045,1.019609,0.934609,1.016709,0.919521,1.014290,0.909084,2.346703,1.401315,1.928693,2.222741
98,1.0,0.972379,0.901844,0.969741,0.910263,0.944868,0.898827,0.942786,0.882623,0.939046,0.905772,0.939133,0.905836,2.293159,0.927965,1.210482,1.355716


In [16]:
df_5_8_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "EBZ Range": each_range_deff_6_list,
        "EBPA(0.3) Range": each_range_deff_3_list,
        "EBPA(0.4) Range": each_range_deff_4_list,
        "EBPA(0.5) Range": each_range_deff_5_list,
    }
)

In [17]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 1.0
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []

quan_sales_list_5 = []
true_quan_sales_list_5 = []

ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    np.random.seed(i + 10)
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # QUAN
    quan_bounds = bound_quan(X, 0.1)
    quan_po_sales, quan_po_prices = predict_optimize(M, X, Y, quan_bounds)
    true_quan_po_sales = np.sum(sales_function(quan_po_prices, alpha, beta))
    quan_sales_list_5.append(quan_po_sales / so_sales)
    true_quan_sales_list_5.append(true_quan_po_sales / so_sales)

    # EBZ
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)

    # EBPA(0.3)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)

    # EBPA(0.4)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)

    # EBPA(0.5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [18]:
df5_10 = pd.DataFrame(
    {
        "SO": so_sales_list_5,
        "PO": po_sales_list_5,
        "True PO": true_po_sales_list_5,
        "QUAN": quan_sales_list_5,
        "True QUAN": true_quan_sales_list_5,
        "EBZ": ebz_po_sales_list_5,
        "True EBZ": true_ebz_po_sales_list_5,
        "EBPA(0.3)": ebpa3_po_sales_list_5,
        "True EBPA(0.3)": true_ebpa3_po_sales_list_5,
        "EBPA(0.4)": ebpa4_po_sales_list_5,
        "True EBPA(0.4)": true_ebpa4_po_sales_list_5,
        "EBPA(0.5)": ebpa5_po_sales_list_5,
        "True EBPA(0.5)": true_ebpa5_po_sales_list_5,
        "EBZ Range": ebz_range_deff_list_5,
        "EBPA(0.3) Range": ebpa3_range_deff_list_5,
        "EBPA(0.4) Range": ebpa4_range_deff_list_5,
        "EBPA(0.5) Range": ebpa5_range_deff_list_5,
    }
)

In [19]:
df_5_10_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "EBZ Range": each_range_deff_6_list,
        "EBPA(0.3) Range": each_range_deff_3_list,
        "EBPA(0.4) Range": each_range_deff_4_list,
        "EBPA(0.5) Range": each_range_deff_5_list,
    }
)

In [20]:
df_5_6.to_csv("df_5_6.csv")
df_5_8.to_csv("df_5_8.csv")
df5_10.to_csv("df5_10.csv")

df_5_6_each.to_csv("df_5_6_each.csv")
df_5_8_each.to_csv("df_5_8_each.csv")
df_5_10_each.to_csv("df_5_10_each.csv")
